In [1]:
import pandas as pd
import requests
import json
import os

In [2]:
# Base folder for data
data_path = '..//data//'

# Folders for city-wide/provincial/regional senatorial election results
results_cty_path = '..//data//results_cty//'
results_prv_path = '..//data//results_prv//'
results_reg_path = '..//data//results_reg//'

# Folder for summarized results datasets
summary_path = '..//data//results_summary//'

In [3]:
# Listing all successfully downloaded data
file_list_cty = os.listdir(results_cty_path)
file_list_prv = os.listdir(results_prv_path)
file_list_reg = os.listdir(results_reg_path)

## Format national results

In [4]:
with open(os.path.join(data_path,'SENATOR_OF_PHILIPPINES.json'),"r") as f:
    nat_json = json.load(f)
nat_entries = nat_json["result"][0]["candidates"]
nat_df = pd.DataFrame(nat_entries)

nat_df[["party_full","party_code"]] = nat_df['party'].str.split("|",n=2,expand=True)
nat_df = nat_df.reset_index().rename(columns={'index':'cand_id'})  # Assigning arbitrary unique ID for each candidate 

In [5]:
# Saving the file
nat_df.to_csv(os.path.join(summary_path,'national.csv'),index=False)
nat_df.drop(columns='vote_count').to_csv(os.path.join(summary_path,'sen_details.csv'),index=False)

## Format regional results

In [6]:
### Conversion dictionary from candidate's name to arbitrarily assigned unique ID
conv_name_id = {x:y for x,y in zip(nat_df['name'],nat_df['cand_id'])}

In [7]:
reg_data = []
reg_det_data = []
counter = 0        # For assigning arbitrary unique ID to each region
for file in file_list_reg:
    with open(os.path.join(results_reg_path,file)) as f:
        temp_json = json.load(f)

    # Summarizing details for each region (region ID, region name, total voters processed)
    reg_det_data.append({'reg_id':counter,'reg_name':temp_json["location_code"],'total_voters_processed':int(temp_json['total_voters_processed'].split("/")[0])})

    # Summarizing results for each region and for each candidate (region ID, candidate ID, vote count)
    reg_data += [{'reg_id':counter,'cand_id':conv_name_id[x['name']],'vote_count':x['vote_count']} \
                 for x in temp_json['result'][0]["candidates"]]
    
    counter+=1

In [8]:
### Conversion to dataframe and saving of the details of the regions and summarized election results
reg_det_df = pd.DataFrame(reg_det_data)
reg_df = pd.DataFrame(reg_data)

reg_det_df.to_csv(os.path.join(summary_path,'region_details.csv'),index=False)
reg_df.to_csv(os.path.join(summary_path,'regional.csv'),index=False)

## Format provincial results

In [9]:
prv_data = []
prv_det_data = []
counter = 0        # For assigning arbitrary unique ID to each province
for file in file_list_prv:
    with open(os.path.join(results_prv_path,file)) as f:
        temp_json = json.load(f)

    # Summarizing details for each province (province ID, province's location code, total voters processed)
    prv_det_data.append({'prv_id':counter,'location_code':temp_json["location_code"],'total_voters_processed':int(temp_json['total_voters_processed'].split("/")[0])})

    # Summarizing results for each province and for each candidate (province ID, candidate ID, vote count)
    prv_data += [{'prv_id':counter,'cand_id':conv_name_id[x['name']],'vote_count':x['vote_count']} \
                 for x in temp_json['result'][0]["candidates"]]
    
    counter+=1

In [10]:
### Conversion to dataframe and saving of the details of the provinces and summarized election results
prv_det_df = pd.DataFrame(prv_det_data)
prv_df = pd.DataFrame(prv_data)

# Further division of location code to region name and province name
prv_det_df[['reg_name','prv_name']] = prv_det_df['location_code'].str.split('|',n=2,expand=True)

prv_det_df.to_csv(os.path.join(summary_path,'province_details.csv'),index=False)
prv_df.to_csv(os.path.join(summary_path,'provincial.csv'),index=False)

## Format municipal/city-wide results

In [11]:
cty_data = []
cty_det_data = []
counter = 0        # For assigning arbitrary unique ID to each city/municipality
for file in file_list_cty:
    with open(os.path.join(results_cty_path,file)) as f:
        temp_json = json.load(f)

    # Summarizing details for each city/mun (city ID, city's location code, total voters processed)
    cty_det_data.append({'cty_id':counter,'location_code':temp_json["location_code"],'total_voters_processed':int(temp_json['total_voters_processed'].split("/")[0])})

    # Summarizing results for each city and for each candidate (city ID, candidate ID, vote count)
    cty_data += [{'cty_id':counter,'cand_id':conv_name_id[x['name']],'vote_count':x['vote_count']} \
                 for x in temp_json['result'][0]["candidates"]]
    
    counter+=1

In [12]:
### Conversion to dataframe and saving of the details of the cities/municipalities and summarized election results
cty_det_df = pd.DataFrame(cty_det_data)
cty_df = pd.DataFrame(cty_data)

# Further division of location code to region name, province name and city name
cty_det_df[['reg_name','prv_name','cty_name']] = cty_det_df['location_code'].str.split('|',n=3,expand=True)

cty_det_df.to_csv(os.path.join(summary_path,'city_details.csv'),index=False)
cty_df.to_csv(os.path.join(summary_path,'city.csv'),index=False)